# Prelims

In [1]:
# -----------------MAGIC COMMANDS---------------
# magic commands to enable autoreload for imported packages
%load_ext autoreload
%autoreload 2


# ------------------INSTALLS--------------------
# # installation of geopy for geolocation
# !pip install geopy


# -------------------IMPORTS---------------------
# basic project organizing stuff
import os

# basic EDA
import ydata_profiling # basic EDA package
import pandas as pd
import numpy as np

# viz
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

# geolocation
from geopy.distance import geodesic

# Data

## Loading data

In [2]:
# see where we are
# os.getcwd()

# create relative path to the data folder
csv_path = os.path.join("..", "data")

# # sanity check
# pd.read_csv(os.path.join(csv_path, 'customers.csv')).head()

# creating list with csv filenames
file_names = [name for name in os.listdir(csv_path) if name[-4:] == ".csv"]

# creating list with desired names
key_names = [name.replace(".csv", "_df") for name in file_names]

# creating dictionary with all the data
data = {key_name: pd.read_csv(os.path.join(csv_path, file)) 
        for (key_name, file) in zip (key_names, file_names)}

In [3]:
data.keys()

dict_keys(['leads_qualified_df', 'order_items_df', 'geolocation_df', 'customers_df', 'order_reviews_df', 'order_payments_df', 'product_category_name_translation_df', 'products_df', 'leads_closed_df', 'orders_df', 'sellers_df'])

### Extracting dataframes from dictionary to shorten code needed

In [9]:
customers = data["customers_df"]
orders = data["orders_df"]
items = data["order_items_df"]
payments = data["order_payments_df"]
reviews = data["order_reviews_df"]
geolocation = data["geolocation_df"]
products = data["products_df"]
product_name_transl = data["product_category_name_translation_df"]
sellers = data["sellers_df"]

# marketing dfs
# leads_qu = data["leads_qualified_df"]
# leads_cl = data["leads_closed_df"]

### Merging product translation to product df

In [10]:
products = pd.merge(products, product_name_transl, how = "left", on = "product_category_name").drop(columns = ["Unnamed: 0_x", "product_category_name", "Unnamed: 0_y"])

## Some basic engineering/cleaning

### Dropping Unnamed columns and duplicates

**Note:** The unnamed columns come from the sql to df conversion done on the other notebook. It just has the indices so it's useless

In [11]:
customers = customers.drop(columns = "Unnamed: 0").drop_duplicates()
orders = orders.drop(columns = "Unnamed: 0").drop_duplicates()
items = items.drop(columns = "Unnamed: 0").drop_duplicates()
payments = payments.drop(columns = "Unnamed: 0").drop_duplicates()
reviews = reviews.drop(columns = "Unnamed: 0").drop_duplicates()
geolocation = geolocation.drop(columns = "Unnamed: 0").drop_duplicates()
sellers = sellers.drop(columns = "Unnamed: 0").drop_duplicates()
products = products.drop_duplicates()

### Averaging longitude and latitude for later convenience

In [12]:
geolocation = geolocation.groupby(by = "geolocation_zip_code_prefix").mean()

### Converting date objects to datetimes and creating delivery timedeltas

In [13]:
# orders
orders[["order_purchase_timestamp", "order_approved_at", 
              "order_delivered_carrier_date", "order_delivered_customer_date", 
              "order_estimated_delivery_date"]] = orders[["order_purchase_timestamp", "order_approved_at", 
              "order_delivered_carrier_date", "order_delivered_customer_date", 
              "order_estimated_delivery_date"]].apply(pd.to_datetime)

# items
items[["shipping_limit_date"]] = items[["shipping_limit_date"]].apply(pd.to_datetime)

# reviews
reviews[["review_creation_date", "review_answer_timestamp"]] = reviews[["review_creation_date", "review_answer_timestamp"]].apply(pd.to_datetime)

In [ ]:
orders["delivery_time"] = (orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]).dt.days
orders["delivery vs estimate"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"]).dt.days
orders["carrier_to_customer"] = (orders["order_delivered_customer_date"] - orders["order_delivered_carrier_date"]).dt.days

### Merging Review Title and Review Comment - Filling nulls with "no review"

In [16]:
# merging review title and comment
reviews["review_all"] = reviews["review_comment_title"].add(reviews["review_comment_message"], fill_value = "")
reviews = reviews.drop(columns = ["review_comment_message", "review_comment_title"])

# filling nulls
reviews.loc[reviews["review_all"].isna(), "review_all"] = "no review"

### Creating Product Volume and dropping separate width, length, height columns, changing mass to Kg

In [17]:
products["volume_litres"] = products["product_length_cm"]*products["product_width_cm"]*products["product_height_cm"]/1000
products["mass_kg"] = products["product_weight_g"]/1000
products.drop(columns = "product_weight_g", inplace = True)

products.drop(columns = ["product_length_cm", "product_height_cm", "product_width_cm"], inplace = True)

# Basic Summary Statistics